### Installing neccessary libraries

In [13]:
# TensorFlow
import tensorflow as tf
import tensorflow.data as tfd
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Others
import os
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.image as implt

In [14]:
# Define AUTOTUNE for TensorFlow dataset
AUTOTUNE = tf.data.experimental.AUTOTUNE

### Data Preprocessing

In [15]:
# Define necessary variables
IMG_WIDTH = 200
IMG_HEIGHT = 50
IMG_SIZE = (IMG_WIDTH, IMG_HEIGHT)
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 1e-3
MODEL_NAME = "CharacterRecognition-Model"
TRAIN_SIZE = BATCH_SIZE * 1500
VALID_SIZE = BATCH_SIZE * 1000
TEST_SIZE  = BATCH_SIZE * 600

# Training callbacks
CALLBACKS = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath=MODEL_NAME + ".h5.keras", save_best_only=True)
]

# Paths
train_csv_path = r'D:\Projects\Handwritten OCR\Dataset\CSV\written_name_train.csv'
valid_csv_path = r'D:\Projects\Handwritten OCR\Dataset\CSV\written_name_validation.csv'
test_csv_path = r'D:\Projects\Handwritten OCR\Dataset\CSV\written_name_test.csv'
train_image_dir = r'D:\Projects\Handwritten OCR\Dataset\train_v2\train'
valid_image_dir = r'D:\Projects\Handwritten OCR\Dataset\validation_v2\validation'
test_image_dir = r'D:\Projects\Handwritten OCR\Dataset\test_v2\test'

# Setup random seeds for numpy and TensorFlow
np.random.seed(2569)
tf.random.set_seed(2569)

In [16]:
# Load the csv flies
train_csv = pd.read_csv(train_csv_path)[:TRAIN_SIZE]
test_csv = pd.read_csv(test_csv_path)[:TEST_SIZE]
valid_csv = pd.read_csv(valid_csv_path)[:VALID_SIZE]

In [17]:
# See how the data looks like
train_csv.head()

,FILENAME,IDENTITY
0,TRAIN_00001.jpg,BALTHAZAR
1,TRAIN_00002.jpg,SIMON
2,TRAIN_00003.jpg,BENES
3,TRAIN_00004.jpg,LA LOVE
4,TRAIN_00005.jpg,DAPHNE


In [18]:
# Get the train labels
train_labels = [str(word) for word in train_csv["IDENTITY"].to_numpy()]

# #Extract all the unique characters
unique_characters = set(char for word in train_labels for char in word)

# Define the number of classes (for labels) based on the number of unique characters
n_classes = len(unique_characters)

print("Number of unique classes (unique characters): ", n_classes)

Number of unique classes (unique characters):  44


In [19]:
# Get the maximum length that a label can have
MAX_LABEL_LENGTH = max(map(len, train_labels))

print("The length of the longest label in the dataset: ", MAX_LABEL_LENGTH)

The length of the longest label in the dataset:  24


In [20]:
train_csv['FILENAME'] = [train_image_dir + f"/{filename}" for filename in train_csv['FILENAME']]
test_csv['FILENAME'] = [test_image_dir + f"/{filename}" for filename in test_csv['FILENAME']]
valid_csv['FILENAME'] = [valid_image_dir + f"/{filename}" for filename in valid_csv['FILENAME']]

In [21]:
train_csv.head()

,FILENAME,IDENTITY
0,D:\Projects\Handwritten OCR\Dataset\train_v2\t...,BALTHAZAR
1,D:\Projects\Handwritten OCR\Dataset\train_v2\t...,SIMON
2,D:\Projects\Handwritten OCR\Dataset\train_v2\t...,BENES
3,D:\Projects\Handwritten OCR\Dataset\train_v2\t...,LA LOVE
4,D:\Projects\Handwritten OCR\Dataset\train_v2\t...,DAPHNE


In [24]:
# Char to num
char_to_num = StringLookup(vocabulary = list(unique_characters), mask_token=None)
num_to_char = StringLookup(vocabulary = char_to_num.get_vocabulary(), mask_token=None, invert=True)

len(char_to_num.get_vocabulary()) + 1

46

In [25]:
'''
This function gets the image path and reads the image using TensorFlow,
then the image will be decoded and will be converted to float datatype.
Next resize and transpose will be applied to it.
In the final step the image will be converted to a Numpy Array using tf.cast
'''


def load_image(image_path):    
    # Read the image
    image = tf.io.read_file(image_path)

    # Decode the image into numerical format uint8
    decoded_image = tf.image.decode_jpeg(contents=image, channels=1)

    # Convert image datatype to float32
    convert_image = tf.image.convert_image_dtype(image=decoded_image, dtype=tf.float32)

    # Resize and transpose
    resized_image = tf.image.resize(images=convert_image, size=IMG_SIZE)
    image = tf.transpose(resized_image, perm=[1, 0, 2])

    # Convert to numpy array (Tensor)
    image_array = tf.cast(image, dtype=tf.float32)

    return image_array

In [26]:
'''
This function takes an image path and label as input and returns the dictionary containing the processed image tensor and the label tensor.
First, it loads the image using the load_image function, that we just made above.
Then it converts the given label string into a sequence of Unicode characters using the unicode_split function.
Next, it uses the char_to_num layer to convert each character in the label to a numerical reperesentation. It pads the numerical representation with a special class (n_classes) to ensure that all the labels have the same length (MAX_LABEL_LENGTH).
Finally, it returns a dictionary containing the processed image tensor and the label tensor.
'''


def encode_single_sample(image_path, label:str):
   # Get the image
   image = load_image(image_path)

   # Convert the label intop characters
   chars = tf.strings.unicode_split(label, input_encoding='UTF-8')

   # Convert the characters into vectors
   vectors = char_to_num(chars)

   # Pad label
   pad_size = MAX_LABEL_LENGTH - tf.shape(vectors)[0]
   vectors = tf.pad(vectors, paddings = [[0, pad_size]], constant_values = n_classes+1)

   return {'image': image, 'label': vectors}

In [29]:
# Training Dataset
train_ds = tfd.Dataset.from_tensor_slices(
    (np.array(train_csv['FILENAME'].to_list()), np.array(train_csv['IDENTITY'].to_list()))
).shuffle(1000).map(encode_single_sample, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Validation Dataset
valid_ds = tfd.Dataset.from_tensor_slices(
    (np.array(valid_csv['FILENAME'].to_list()), np.array(valid_csv['IDENTITY'].to_list()))
).shuffle(1000).map(encode_single_sample, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

# Test Dataset
test_ds = tfd.Dataset.from_tensor_slices(
    (np.array(test_csv['FILENAME'].to_list()), np.array(test_csv['IDENTITY'].to_list()))
).shuffle(1000).map(encode_single_sample, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [30]:
# Check the data redistribution
print("Training Data Size   : ", tfd.Dataset.cardinality(train_ds).numpy() * BATCH_SIZE)
print("Validation Data Size : ", tfd.Dataset.cardinality(valid_ds).numpy() * BATCH_SIZE)
print("Test Data Size       : ", tfd.Dataset.cardinality(test_ds).numpy() * BATCH_SIZE)

Training Data Size   :  24000
Validation Data Size :  16000
Test Data Size       :  9600
